# Verify that absolute frequency $\omega$ is constant

Velocity field 

$$\mathbf{U} = (U(x),0,0),$$
which is a steady current only varying  distance. Alternatively

$$\mathbf{U} = (U(x-ct),0,0),$$
which is a steady current propagating in a frame of reference moving with velocity $C$.


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import cmocean
import xarray as xa
import pytest


import sys,os
testdir = os.path.dirname(os.getcwd() + '/')
srcdir = '..'
sys.path.insert(0, os.path.abspath(os.path.join(testdir, srcdir)))

from ocean_wave_tracing import Wave_tracing


#%matplotlib inline

In [8]:
from ocean_wave_tracing.util_solvers import ForwardEuler

In [9]:
ncin = xa.open_dataset('../notebooks/idealized_input.nc')

In [10]:
U = ncin.U_zero.isel(time=0).copy()
U[:,20::] = -1


In [11]:
X = ncin.x.data
Y = ncin.y.data
nx = len(X)
ny = len(Y)
dx=dy=X[1]-X[0]
nb_wave_rays = 10#550#nx
T = 1000
nt = 200#500 # 1500
wave_period = 10 #s
X0, XN = X[0], X[-1] 
Y0, YN = Y[0], Y[-1]

i_w_side = 'left'
theta0=0
idt0=0


In [12]:
omega_x = []
X_coor = []
C = []
Del = []
kk = []
ddt = []
for nt in [10,20,50,75,100,200,300,500,700]:
    wt = Wave_tracing(U, ncin.V_zero.isel(time=0), 
    #wt = Wave_tracing(ncin.U.isel(time=7), ncin.V.isel(time=7), 
                        nx, ny, nt,T,dx,dy, nb_wave_rays=nb_wave_rays,
                        domain_X0=X0, domain_XN=XN,
                        domain_Y0=Y0, domain_YN=YN,
                         )
    wt.set_initial_condition(wave_period, theta0,incoming_wave_side=i_w_side)
    wt.solve()
    
    courant = wt.C
    omega_field = wt.sigma(wt.ray_k,wt.ray_depth) + wt.ray_kx*wt.ray_U + wt.ray_ky*wt.ray_V
    Delta = ((omega_field[5,0]-omega_field[5,-2])/omega_field[5,0])*100
    print(courant, Delta)
    
    omega_x.append(omega_field)
    X_coor.append(wt.ray_x[4,:])
    C.append(courant)
    Del.append(Delta)
    kk.append(wt.ray_k[4,:])
    ddt.append(wt.dt)



8.806549958657467 6.404877988970008
4.403274979328733 6.404877988970008
1.7613099917314934 2.080305227895062
1.174206661154329 0.5428860926004061
0.8806549958657467 -0.24546955532181963
0.44032749793287335 0.8507042024007319
0.29355166528858223 -0.3125063843620607
0.17613099917314934 0.13586044107945272
0.12580785655224952 -0.0015814090866245211


In [16]:
fs=17

fig,ax = plt.subplots(figsize=(14,5))
colors = cmocean.cm.thermal(np.linspace(0,1,len(X_coor)))
for i in range(len(X_coor)):
    y = omega_x[i][3][0:-1]/ omega_x[i][3][0]
    ax.plot(X_coor[i][0:-1],y,
            '-o',
            label=r'C={}, $\Delta=${}%'.format(np.round(C[i],2), np.round(np.abs(Del[i]),2)),
            c=colors[i],
           lw=2,
           alpha=0.8)
    
ax.legend(fontsize=fs-6)
ax.grid()
ax.set_ylabel(r'$\omega/\omega_{0}$',fontsize=fs)
ax.set_xlabel(r'$X$[m]',fontsize=fs)
ax.set_xlim([1000,3000])
ax.set_ylim([0.932,1.033])

ax.axvline(wt.x[19],c='tab:green',ls='--',lw=4,alpha=0.4)
ax.axvline(wt.x[20],c='tab:green',ls='--',lw=4,alpha=0.4)
ax.fill_between(wt.x[20::],0,2,color='tab:green',alpha=0.4)


ax.text(wt.x[11],1.025,r'U=0, $\omega=\omega_0$',fontsize=fs-3,c='k',horizontalalignment='center')
ax.text(wt.x[29],1.025,r'U=-1',fontsize=fs-3,c='k',horizontalalignment='center')
ax.text(wt.x[20],1.035,r'U=-1',fontsize=fs-3,c='k',horizontalalignment='center')
ax.text(wt.x[19],1.035,r'U=0',fontsize=fs-3,c='k',horizontalalignment='center')



fig.tight_layout()
#fig.savefig('numerical_convergence_RK4.png',dpi=120)
plt.close(fig)

## Inspecting the change in $k$

In [18]:
k0 = wt.ray_k[0,0]
k1 = (k0*(2*wt.ray_cg[0,0]))/(wt.U[0,-1,-1].values + (2*wt.ray_cg[0,-2]))

fig,ax = plt.subplots(figsize=(14,5))
colors = cmocean.cm.thermal(np.linspace(0,1,len(X_coor)))



for i in range(len(X_coor)):
    #y = omega_x[i][3][0:-1]/ omega_x[i][3][0]
    ax.plot(X_coor[i][0:-1],kk[i][0:-1]/k0,
            '-o',
            label=r'C={}, $\Delta=${}%'.format(np.round(C[i],2), np.round(np.abs(Del[i]),2)),
            c=colors[i],
           lw=2,
           alpha=0.8)
    
ax.legend(fontsize=fs-5,loc='lower right')
ax.grid()
ax.set_ylabel(r'$k/k_{0}$',fontsize=fs)
ax.set_xlabel(r'$X$[m]',fontsize=fs)
ax.set_xlim([1000,3000])
ax.set_ylim([0.95,1.2])



ax.axvline(wt.x[19],c='tab:green',ls='--',lw=4,alpha=0.4)
ax.axvline(wt.x[20],c='tab:green',ls='--',lw=4,alpha=0.4)

ax.axhline(k1/k0,c='k',ls='--',lw=4,alpha=0.4)
ax.fill_between(wt.x[20::],0,2,color='tab:green',alpha=0.5)

ax.text(wt.x[20],1.205,r'U=-1',fontsize=fs-3,c='k',horizontalalignment='center')
ax.text(wt.x[19],1.205,r'U=0',fontsize=fs-3,c='k',horizontalalignment='center')

ax.text(wt.x[11]+30,1.16,r'$k_{an}$ for U=-1',fontsize=fs-3,c='k',horizontalalignment='center')

fig.tight_layout()
#fig.savefig('numerical_convergence_for_k_RK4.png',dpi=120)
plt.close(fig)

## Compute $\omega$

$$\omega = \sigma + \mathbf{k}\cdot\mathbf{U}$$

In [19]:
ray_id = 4
omega = wt.sigma(wt.ray_k[ray_id,:],wt.ray_depth[ray_id,:]) + wt.ray_kx[ray_id,:]*wt.ray_U[ray_id,:] + wt.ray_ky[ray_id,:]*wt.ray_V[ray_id,:]
omega_field = wt.sigma(wt.ray_k,wt.ray_depth) + wt.ray_kx*wt.ray_U + wt.ray_ky*wt.ray_V

print(((omega[0]-omega[-2])/omega[0])*100)
print(((omega_field[5,0]-omega_field[5,-2])/omega_field[5,0])*100)

-0.0015814090866245211
-0.0015814090866245211
